## Table of Contents

<ul>
    <li><a href='#Dependencies'>Dependencies</a></li>
    </li>
    <li><a href='#Modelling'>Modelling</a>
        <ol style='margin-top: 0em;'>
            <li><a href='#1.-State-Variables'>State Variables</a></li>
            <li><a href='#2.-System-Parameters'>System Parameters</a></li>
            <li><a href='#3.-Policy-Functions'>Policy Functions</a></li>
            <li><a href='#4.-State-Update-Functions'>State Update Functions</a></li>
            <li><a href='#5.-Partial-State-Update-Blocks'>Partial State Update Blocks</a></li>
        </ol>
    </li>
    <li><a href='#Simulation'>Simulation</a>
        <ul style='margin-top: 0em; start="6"'>
            <li><a href='#6,-7,-and-8:-Configuration,-Execution-and-Output-Preparation'>6, 7, and 8: Configuration, Execution and Output Preparation</a></li>
            <li><a href='#9.-Analysis'>Analysis</a></li>
        </ul>
    </li>
</ul>

---

# Dependencies

In [ ]:
# No need for cadCAD standard dependencies
# we're going to use the cadCAD_tools library
# Install it through `pip install cadCAD-tools`

from cadCAD_tools import profile_run

In [ ]:
# Additional dependencies

# For visualizing how much each substep is impacting the simulation
from cadCAD_tools.profiling.visualizations import visualize_substep_impact

# For visualizing how much elapsed time takes for each timestep
from cadCAD_tools.profiling.visualizations import visualize_elapsed_time_per_ts

# For analytics
import pandas as pd
import numpy as np
# For visualization
import plotly.express as px

# Modelling

## 1. State Variables

In [ ]:
initial_state = {
    'predator_population': 15, # Initial number of predators
    'prey_population': 100, # Initial number of preys
}
initial_state

## 2. System Parameters

In [ ]:
system_params = {
    # Parameters describing the interaction between the two populations:
    
    # A parameter used to calculate the rate of predator birth
    "predator_birth_parameter": [0.01], 
    # A parameter used to calculate the rate of predator death
    "predator_death_parameter": [1.0],
    # A parameter used to calculate the rate of prey birth
    "prey_birth_parameter": [0.6, 1.0],
    # A parameter used to calculate the rate of prey death
    "prey_death_parameter": [0.03],

    # Parameters used for random Numpy variable tuning
    # These parameters scale the random variable used to create the random birth / death rate
    
    "random_predator_birth": [0.0002],
    "random_predator_death": [0.005],
    "random_prey_birth": [0.1],
    "random_prey_death": [0.1],

    # Parameter used as conversion factor between 1 unit of time and 1 timestep
    # 10 timesteps == 1 unit of time, i.e. every 10 cadCAD model timesteps, 1 unit of actual model time passes
    
    "dt": [0.1],
}

## 3. Policy Functions

In [ ]:
def p_predator_births(params, substep, state_history, previous_state):
    '''Predator Births Policy Function
    The predator birth rate (rate of predators born per unit of time) is a product of
    the prey population and the predator birth parameter plus a random variable.
    
    i.e. the larger the prey population, the higher the predator birth rate
    '''
    # Parameters   
    dt = params['dt']
    predator_birth_parameter = params['predator_birth_parameter']
    random_predator_birth = params['random_predator_birth']

    # State Variables
    predator_population = previous_state['predator_population']
    prey_population = previous_state['prey_population']

    # Calculate the predator birth rate
    birth_rate = prey_population * (predator_birth_parameter + np.random.random() * random_predator_birth)
    # Calculate change in predator population
    births = birth_rate * predator_population * dt

    return {'add_to_predator_population': births}


def p_predator_deaths(params, substep, state_history, previous_state):
    '''Predator Deaths Policy Function
    The predator death rate (rate of predators that die per unit of time) is a function of
    the predator death parameter plus a random variable.
    
    i.e. the larger the predator death parameter, the higher the predator death rate
    '''
    # Parameters
    dt = params['dt']
    predator_death_parameter = params['predator_death_parameter']
    random_predator_death = params['random_predator_death']

    # State Variables
    predator_population = previous_state['predator_population']

    # Calculate the predator death rate
    death_rate = predator_death_parameter + np.random.random() * random_predator_death
    # Calculate change in predator population
    deaths = death_rate * predator_population * dt

    return {'add_to_predator_population': -1.0 * deaths}


def p_prey_births(params, substep, state_history, previous_state):
    '''Prey Births Policy Function
    The prey birth rate (rate of preys born per unit of time) is a function of
    the prey birth parameter plus a random variable.
    
    i.e. the larger the prey birth parameter, the higher the prey birth rate
    '''
    # Parameters
    dt = params['dt']
    prey_birth_parameter = params['prey_birth_parameter']
    random_prey_birth = params['random_prey_birth']

    # State Variables
    prey_population = previous_state['prey_population']

    # Calculate the prey birth rate
    birth_rate = prey_birth_parameter + np.random.random() * random_prey_birth
    # Calculate change in prey population
    births = birth_rate * prey_population * dt

    return {'add_to_prey_population': births}


def p_prey_deaths(params, substep, state_history, previous_state):
    '''Prey Deaths Policy Function
    The prey death rate (rate of preys that die per unit of time) is a product of
    the predator population and the prey death parameter plus a random variable.
    
    i.e. the larger the predator population, the higher the prey death rate
    '''
    # Parameters
    dt = params['dt']
    prey_death_parameter = params['prey_death_parameter']
    random_prey_death = params['random_prey_death']

    # State Variables
    prey_population = previous_state['prey_population']
    predator_population = previous_state['predator_population']

    # Calculate the prey death rate
    death_rate = predator_population * (prey_death_parameter + np.random.random() * random_prey_death)
    # Calculate change in prey population
    deaths = death_rate * prey_population * dt

    return {'add_to_prey_population': -1.0 * deaths}

## 4. State Update Functions

In [ ]:
def s_predator_population(params, substep, state_history, previous_state, policy_input):
    '''Predator Population State Update Function
    Take the Policy Input `add_to_predator_population`
    (the net predator births and deaths)
    and add to the `predator_population` State Variable.
    '''
    # Policy Inputs
    add_to_predator_population = policy_input['add_to_predator_population']

    # State Variables
    predator_population = previous_state['predator_population']

    # Calculate updated predator population
    updated_predator_population = predator_population + add_to_predator_population

    return 'predator_population', updated_predator_population


def s_prey_population(params, substep, state_history, previous_state, policy_input):
    '''Prey Population State Update Function
    Take the Policy Input `add_to_prey_population`
    (the net prey births and deaths)
    and add to the `prey_population` State Variable.
    '''
    # Policy Inputs
    add_to_prey_population = policy_input['add_to_prey_population']

    # State Variables
    prey_population = previous_state['prey_population']

    # Calculate updated prey population
    updated_prey_population = prey_population + add_to_prey_population

    return 'prey_population', updated_prey_population

## 5. Partial State Update Blocks

In [ ]:
partial_state_update_blocks = [
    {   
        'label': "Lotka-Volterra equations",
        # Configure the model Policy Functions
        'policies': {
            # Calculate the predator birth rate and number of births
            'predator_births': p_predator_births,
            # Calculate the predator death rate and number of deaths
            'predator_deaths': p_predator_deaths,
            # Calculate the prey birth rate and number of births
            'prey_births': p_prey_births,
            # Calculate the prey death rate and number of deaths
            'prey_deaths': p_prey_deaths,
        },
        # Configure the model State Update Functions
        'variables': {
            # Update the predator population
            'predator_population': s_prey_population,
            # Update the prey population
            'prey_population': s_predator_population
        }
    },
    {
        'label': 'Do nothing',
        'policies': {},
        'variables': {}
    }
]

# Simulation

## 6, 7, and 8: Configuration, Execution and Output Preparation

In [ ]:
df = profile_run(initial_state,
                 system_params,
                 partial_state_update_blocks,
                 20, # 20 timesteps
                 5, # 5 MC runs
                 use_label=True, # Assign PSUB labels to the results df
                 assign_params=True # Have the parameters on the results df
                )

## 9. Analysis

In [ ]:
# See how much each substep is slowing the sim down.
# Pass relative=True to see the fraction
# relative=False will give the absolute time
visualize_substep_impact(df, relative=True)

In [ ]:
# See how much time has elapsed for each timestep
# Pass relative=True to see how much time it took for just one timestep
# relative=False will give the total elapsed time
visualize_elapsed_time_per_ts(df, relative=False)